# Data Science with Python final project

### Question: How popular will a new track probably be given a selected amount of attributes

Why is this interesting?

When you know how popular a new track will be, you can make better decisions regarding advertisment budgets, cost for the music rights etc.

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, glob
import json
import sqlite3 as sl
from sqlalchemy import create_engine
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import text

## Loading Data from DB to DataFrames

#### Tables tracks, lyrics and features provide information about the tracks which could be useful to predict the popularity

In [2]:
engine = create_engine("sqlite:///spotify.db")
tracks_df = pd.read_sql_table("tracks", engine)
lyrics_df = pd.read_sql_table("lyrics", engine)
features_df = pd.read_sql_table("features", engine)

### Check how to do the JOIN

In [3]:
print(f"DataFrame tracks_df has {tracks_df.shape[0]} rows.")
print(f"DataFrame lyrics_df has {lyrics_df.shape[0]} rows.")
print(f"DataFrame features_df has {features_df.shape[0]} rows.")

DataFrame tracks_df has 301508 rows.
DataFrame lyrics_df has 183320 rows.
DataFrame features_df has 301545 rows.


tracks_df and features_df have almost the same amount of rows, but lyrics_df way less
-> check what extra information lyrics_df provides and if we could fill the missing data for the relevant columns

In [4]:
lyrics_df.head()

,uri,title,artist,language,lyrics
0,spotify:track:6Nr3QjImPUhCpUgnykO7hm,Fly In,Lil Wayne,en,"[Intro]\nSo they ask me\n""Young boy\nWhat you ..."
1,spotify:track:4P7VFiaZb3xrXoqGwZXC3J,Lollipop Remix,Lil Wayne,en,[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (You...
2,spotify:track:4T90fwS4tw1Qlxige4a9mK,Lollipop Remix,Lil Wayne,en,[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (You...
3,spotify:track:0s5bCEmMYwttQIZLHLwOF3,Lollipop Remix,Lil Wayne,en,[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (You...
4,spotify:track:6Y6f7LSvHxUA61ItYiSMKE,Money On My Mind,Lil Wayne,en,"[Intro]\nYeah\nMoney on my mind, money on my m..."


Only relevant column is the language column, which we could fill up by what other languages the Artist has used

### -> Using OUTERJOIN

In [5]:
df = tracks_df.merge(lyrics_df, how="outer", on="uri")
df = df.merge(features_df, how="outer", on="uri")
df.set_index("uri", inplace=True)

In [6]:
df.head(10)

,type_x,name,duration_ms_x,popularity,explicit,is_local,available_markets,preview_url,disc_number,track_number,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0ltRc18CGfqQXVoh9HmJEU,track,There'll Come A Day,203213.0,33.0,0.0,0.0,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",None,1.0,5.0,...,0.802,0.000000,4.0,0.1930,-4.046,1.0,0.0411,159.017,4.0,0.353
spotify:track:6ZOPMDgEU9gPg0gtlgTc3E,track,Alive,193125.0,17.0,0.0,0.0,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",https://p.scdn.co/mp3-preview/ced485344d371879...,1.0,1.0,...,0.830,0.000000,8.0,0.1680,-4.448,0.0,0.0837,127.901,4.0,0.221
spotify:track:28Wu2b54s7TPn0o8xSteeQ,track,Kill Em With Kindness - Felix Cartal Remix,196293.0,0.0,0.0,0.0,,None,1.0,1.0,...,0.774,0.000005,10.0,0.2530,-2.886,0.0,0.0365,121.970,4.0,0.503
spotify:track:20wgVaEsW9ElKWEZkZYOrm,track,Te Vas,169346.0,0.0,0.0,0.0,,None,1.0,10.0,...,0.527,0.000000,5.0,0.1750,-7.995,0.0,0.1090,90.043,4.0,0.492
spotify:track:4AZlgqkThHYfPSstdVuy2f,track,Goin' Back Down,225740.0,0.0,0.0,0.0,,None,1.0,3.0,...,0.877,0.000013,10.0,0.3070,-4.924,0.0,0.1060,170.211,4.0,0.632
spotify:track:16cCeDTKDk4MAMIjhJJMoq,track,Waiting For Your Love,248255.0,12.0,0.0,0.0,"AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, CY...",https://p.scdn.co/mp3-preview/e0d2b706435ec324...,1.0,1.0,...,0.766,0.012700,0.0,0.1080,-5.717,1.0,0.0341,121.061,4.0,0.457
spotify:track:23NWg3ow2UrMun7zRXHfce,track,Good Together,195486.0,0.0,0.0,0.0,,None,1.0,1.0,...,0.623,0.000000,1.0,0.4920,-7.006,1.0,0.1050,104.035,4.0,0.495
spotify:track:0Rg6WHOKKrcpVsJeWKmbmZ,track,悪いことばかり学んで,222533.0,5.0,0.0,0.0,JP,https://p.scdn.co/mp3-preview/12c525da8d91f462...,1.0,3.0,...,0.879,0.000000,9.0,0.2720,-2.850,0.0,0.0590,79.997,4.0,0.708
spotify:track:309TfpIu0xvJVFQuYjThq8,track,Drunk,176243.0,0.0,0.0,0.0,,None,1.0,6.0,...,0.302,0.000034,3.0,0.0989,-6.020,1.0,0.0304,87.827,4.0,0.238


## Data transformation

### Deleting duplicated columns

##### title and name have the same meaning
-> check which one has more NaN values and delete it

In [7]:
print(f"title has {df['title'].isnull().sum()} NaN values.")
print(f"name has {df['name'].isnull().sum()} NaN values.")

title has 118226 NaN values.
name has 38 NaN values.


In [8]:
del df["title"]

### Fixing language

In [9]:
print(f"{df['language'].isnull().sum()/df.shape[0]*100//1}% of the language entries are NaN.")

40.0% of the language entries are NaN.


##### Can we fill it up over the artist?

In [10]:
print(f'{df[df["language"].isna()]["artist"].isnull().sum()/df["language"].isnull().sum()*100//1}% of the rows with missing language have a missing artist value.')

96.0% of the rows with missing language have a missing artist value.


-> No, that won't be a solution

##### Can we fill it up over the language of the language of the name?

How many and what languages do we have?

In [11]:
languages = df["language"].unique()
print(f"There are {languages.shape[0]} unique languages in the language column.")
print("The Languages are: (language/number of tracks in this language)")
for language in languages:
    print(str(language) + ": " + str(df[df["language"] == language].shape[0]), end=" ")

There are 50 unique languages in the language column.
The Languages are: (language/number of tracks in this language)
nan: 0 en: 163995 ja: 74 pt: 909 None: 0 es: 11155 fil: 379 fr: 836 it: 338 fi: 36 de: 376 ko: 417 is: 39 tr: 123 da: 28 sv: 148 no: 19 uk: 1 hi: 56 ar: 40 ru: 32 ta: 15 he: 41 pl: 18 gl: 1 id: 15 zh: 13 nl: 22 vi: 1 el: 24 fa: 13 la: 6 ro: 6 cy: 2 af: 7 ca: 7 ur: 1 sq: 3 pa: 3 te: 2 ceb: 1 bg: 2 eo: 1 ga: 6 hr: 1 eu: 1 lt: 1 hu: 2 gd: 1 sk: 1 

##### Building classification model to fill up the nan values in the language column

In [12]:
### TODO

##### Check if we can use a language classification model to detect the language from the name

In [14]:
base_options = python.BaseOptions(model_asset_path="language_detector.tflite")
options = text.LanguageDetectorOptions(base_options=base_options)

In [25]:
with python.text.LanguageDetector.create_from_options(options) as detector:
    number_all = df[~df["language"].isnull()].shape[0]
    number_right_detected = 0
    number_not_detected = 0
    for index, row in df[~df["language"].isnull()].iterrows():
        detection_result = detector.detect(str(row["name"]))
        if len(detection_result.detections) > 0 and str(row["language"]) == str(detection_result.detections[0].language_code):
            number_right_detected += 1
        else:
            number_not_detected += 1
    print(f"Number of correct detected languages: {number_right_detected/number_all*100//1}")
    print(f"Number of not detected languages: {number_not_detected/number_all*100//1}")

Number of correct detected languages: 85.0
Number of not detected languages: 14.0
